## Manifest and Metadata Creation

In [28]:
import pandas as pd 
import numpy as np

In [31]:
manifest = pd.read_csv('ref_sugar_sample.csv')
print(manifest.shape)
manifest.columns

(279, 10)


Index(['Unnamed: 0', 'sample_name', 'age_years', 'age_cat', 'sex',
       'specialized_diet_exclude_dairy',
       'specialized_diet_exclude_refined_sugars', 'body_site',
       'specialized_diet_i_do_not_eat_a_specialized_diet',
       'specialized_diet_paleodiet_or_primal_diet'],
      dtype='object')

In [32]:
sample_no_fastq = ['10317.000051146',
 '10317.000058813',
 '10317.000058966',
 '10317.000059165',
 '10317.000066514',
 '10317.000110719',
 '10317.000110720',
 '10317.000110721',
 '10317.000110722']

In [33]:
manifest = manifest.loc[~manifest.sample_name.isin(sample_no_fastq)]

In [34]:
manifest

,Unnamed: 0,sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet
0,6610,10317.000023,69,60s,female,No,Yes,UBERON:feces,No,No
1,6809,10317.000024,59,50s,male,No,Yes,UBERON:feces,No,No
2,8522,10317.000030,27,20s,male,No,Yes,UBERON:feces,No,Yes
3,9440,10317.000033,60,60s,female,No,Yes,UBERON:feces,No,No
4,9804,10317.000035,41,40s,female,No,Yes,UBERON:feces,No,Yes
...,...,...,...,...,...,...,...,...,...,...
274,25831,10317.000113,38,30s,male,No,Yes,UBERON:feces,Yes,No
275,25834,10317.000113,49,40s,female,No,Yes,UBERON:feces,No,No
276,26079,10317.000114,55,50s,male,No,Yes,UBERON:feces,No,No
277,26116,10317.000114,22,20s,male,No,Yes,UBERON:feces,No,No


In [35]:
print(manifest.sample_name.nunique())

279


In [39]:
run_ids = pd.read_csv('fastq_goodness.csv')
run_ids.head(5)
ids = run_ids['fastq_file']
print(run_ids['fastq_file'].nunique())

268


In [37]:
ids

0      ERR1842797.fastq
1      ERR1842808.fastq
2      ERR1842816.fastq
3      ERR1842877.fastq
4      ERR2032284.fastq
             ...       
263    ERR5337870.fastq
264    ERR5342182.fastq
265    ERR5342208.fastq
266    ERR5342218.fastq
267    ERR5342289.fastq
Name: fastq_file, Length: 268, dtype: object

In [38]:
manifest['sample-id'] = manifest.sample_name

print(manifest.shape)

# Costruiamo e teniamo solo le colonne che ci servono
manifest['run_ids'] = ids
manifest['absolute-filepath'] = '/c/Users/adm/Desktop/Bioscience/gut-microbiota-diet/fastq/goodReads_20/good_20_'+ manifest['run_ids']
manifest['direction'] = 'forward'

manifest = manifest[['sample-id', 'absolute-filepath', 'direction']]

# Scriviamo il csv
manifest.to_csv('fastq_manifest.csv', sep=',', index=False)
manifest

(279, 11)


,sample-id,absolute-filepath,direction
0,10317.000023,/c/Users/adm/Desktop/Bioscience/gut-microbiota...,forward
1,10317.000024,/c/Users/adm/Desktop/Bioscience/gut-microbiota...,forward
2,10317.000030,/c/Users/adm/Desktop/Bioscience/gut-microbiota...,forward
3,10317.000033,/c/Users/adm/Desktop/Bioscience/gut-microbiota...,forward
4,10317.000035,/c/Users/adm/Desktop/Bioscience/gut-microbiota...,forward
...,...,...,...
274,10317.000113,NaN,forward
275,10317.000113,NaN,forward
276,10317.000114,NaN,forward
277,10317.000114,NaN,forward


In [42]:
manifest = manifest.iloc[:267]

## Metadata file creation

In [44]:
data = pd.read_csv('ref_sugar_sample.csv')

sample_name,age_years,age_cat,sex,specialized_diet_exclude_dairy,specialized_diet_exclude_refined_sugars,body_site,specialized_diet_i_do_not_eat_a_specialized_diet,specialized_diet_paleodiet_or_primal_diet

In [46]:
# Aggiungiamo al manifest le informazioni riguardo l'età, il sesso e l'uso degli antibiotici
df_metadata = pd.merge(manifest['sample-id'], 
                       data[['sample_name', 'sex', 'age_cat', 'specialized_diet_exclude_refined_sugars']], 
                       how='inner', 
                       left_on='sample-id', 
                       right_on = 'sample_name')

df_metadata = df_metadata.drop(['sample_name'], axis=1)

import numpy as np

#conditions = [
#    (df_metadata['antibiotic_history']=='Year'),
#    (df_metadata['antibiotic_history']=='6 months'),
#    (df_metadata['antibiotic_history']=='Month'),
#    (df_metadata['antibiotic_history']=='Week'),
#]

#choices = [
#    'B - Year',
#    'C - 6 months',
#    'D - Month',
#    'E - Week']

#df_metadata['antibiotic_history_ord'] = np.select(conditions, choices, default=' A - I have not taken antibiotics in the past year.')
               
# Scrviamo il tsv
df_metadata.to_csv('sample_metadata.tsv', sep='\t', index=False)
df_metadata

,sample-id,sex,age_cat,specialized_diet_exclude_refined_sugars
0,10317.000023,female,60s,Yes
1,10317.000024,male,50s,Yes
2,10317.000030,male,20s,Yes
3,10317.000033,female,60s,Yes
4,10317.000035,female,40s,Yes
...,...,...,...,...
262,10317.000110,male,60s,Yes
263,10317.000110,male,70+,Yes
264,10317.000110,male,60s,Yes
265,10317.000110,male,70+,Yes


In [6]:
%conda update conda
%conda install wget

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - wget

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [4]:
%wget https://data.qiime2.org/distro/core/qiime2-2023.2-py38-linux-conda.yml
%conda env create -n qiime2-2023.2 --file qiime2-2023.2-py38-linux-conda.yml
%conda activate qiime2-2023.2

UsageError: Line magic function `%wget` not found.


In [5]:
%qiime --help

UsageError: Line magic function `%qiime` not found.


In [ ]:
from qiime2 import Artifact
artifact_data = Artifact.import_data('SampleData[SequencesWithQuality]',
                                     '/c/Users/adm/Desktop/Bioscience/gut-microbiota-diet/fastq_manifest.csv',
                                     view_type='SingleEndFastqManifestPhred33')

artifact_data.save('artifact_data.qza')

In [ ]:
artifact_data = Artifact.load('artifact_data.qza')
artifact_data
